<img style="float: right;" src="External%20Causes%20of%20Death2.png">


In [1]:
import numpy as np
import pandas as pd
import pyspark.sql.functions as func
from pyspark.sql.types import IntegerType
from pyspark.sql import SparkSession

In [2]:
spark=SparkSession.builder.master("local").appName("externaldeathcauses").getOrCreate()

In [3]:
#por algun motivo, localmente con spark no pude leer todos los archivos de una, con lo cual voy a tener que loopearlos.
i=1
while i<6:
    df_temp = spark.read.csv(f"data/Morticd10_part{i}.csv",header=True, inferSchema=True)
    df_temp.createOrReplaceTempView(f"temp{i}")
    i=i+1


In [4]:
query = """
    select * from temp1
    union all 
    select * from temp2
    union all
    select * from temp3
    union all
    select * from temp4
    union all
    select * from temp5
    """

df = spark.sql(query)

In [5]:
df.count()

4506221

In [6]:
df.printSchema()

root
 |-- Country: integer (nullable = true)
 |-- Admin1: string (nullable = true)
 |-- SubDiv: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- List: string (nullable = true)
 |-- Cause: string (nullable = true)
 |-- Sex: integer (nullable = true)
 |-- Frmat: integer (nullable = true)
 |-- IM_Frmat: integer (nullable = true)
 |-- Deaths1: integer (nullable = true)
 |-- Deaths2: integer (nullable = true)
 |-- Deaths3: integer (nullable = true)
 |-- Deaths4: integer (nullable = true)
 |-- Deaths5: integer (nullable = true)
 |-- Deaths6: integer (nullable = true)
 |-- Deaths7: integer (nullable = true)
 |-- Deaths8: integer (nullable = true)
 |-- Deaths9: integer (nullable = true)
 |-- Deaths10: integer (nullable = true)
 |-- Deaths11: integer (nullable = true)
 |-- Deaths12: integer (nullable = true)
 |-- Deaths13: integer (nullable = true)
 |-- Deaths14: integer (nullable = true)
 |-- Deaths15: integer (nullable = true)
 |-- Deaths16: integer (nullable = true)
 |-- De

In [7]:
df.columns

['Country',
 'Admin1',
 'SubDiv',
 'Year',
 'List',
 'Cause',
 'Sex',
 'Frmat',
 'IM_Frmat',
 'Deaths1',
 'Deaths2',
 'Deaths3',
 'Deaths4',
 'Deaths5',
 'Deaths6',
 'Deaths7',
 'Deaths8',
 'Deaths9',
 'Deaths10',
 'Deaths11',
 'Deaths12',
 'Deaths13',
 'Deaths14',
 'Deaths15',
 'Deaths16',
 'Deaths17',
 'Deaths18',
 'Deaths19',
 'Deaths20',
 'Deaths21',
 'Deaths22',
 'Deaths23',
 'Deaths24',
 'Deaths25',
 'Deaths26',
 'IM_Deaths1',
 'IM_Deaths2',
 'IM_Deaths3',
 'IM_Deaths4']

In [8]:
df.createOrReplaceTempView("mortality")
df.show(10)

+-------+------+------+----+----+-----+---+-----+--------+-------+-------+-------+-------+-------+-------+-------+-------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+----------+----------+----------+----------+
|Country|Admin1|SubDiv|Year|List|Cause|Sex|Frmat|IM_Frmat|Deaths1|Deaths2|Deaths3|Deaths4|Deaths5|Deaths6|Deaths7|Deaths8|Deaths9|Deaths10|Deaths11|Deaths12|Deaths13|Deaths14|Deaths15|Deaths16|Deaths17|Deaths18|Deaths19|Deaths20|Deaths21|Deaths22|Deaths23|Deaths24|Deaths25|Deaths26|IM_Deaths1|IM_Deaths2|IM_Deaths3|IM_Deaths4|
+-------+------+------+----+----+-----+---+-----+--------+-------+-------+-------+-------+-------+-------+-------+-------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+----------+----------+----------+----------+
|   1400|  null|

Country and disease causes codes to join our initial data 

In [9]:
df_country = spark.read.csv(f"data/country_codes.csv",header=True, inferSchema=True)
df_country.createOrReplaceTempView("country_codes")
df_country.show(10)

+-------+--------------------+
|country|                name|
+-------+--------------------+
|   1010|             Algeria|
|   1020|              Angola|
|   1025|               Benin|
|   1030|            Botswana|
|   1035|        Burkina Faso|
|   1040|             Burundi|
|   1045|            Cameroon|
|   1060|          Cape Verde|
|   1070|Central African R...|
|   1080|                Chad|
+-------+--------------------+
only showing top 10 rows



I need to filter and unpivot the datasource for a more confortable use

In [10]:
query = """
select 
    year,
    sex,
    country,
    cause,
    stack(30,
            'Deaths1',Deaths1,
            'Deaths2',Deaths2,
            'Deaths3',Deaths3,
            'Deaths4',Deaths4,
            'Deaths5',Deaths5,
            'Deaths6',Deaths6,
            'Deaths7',Deaths7,
            'Deaths8',Deaths8,
            'Deaths9',Deaths9,
            'Deaths10',Deaths10,
            'Deaths11',Deaths11,
            'Deaths12',Deaths12,
            'Deaths13',Deaths13,
            'Deaths14',Deaths14,
            'Deaths15',Deaths15,
            'Deaths16',Deaths16,
            'Deaths17',Deaths17,
            'Deaths18',Deaths18,
            'Deaths19',Deaths19,
            'Deaths20',Deaths20,
            'Deaths21',Deaths21,
            'Deaths22',Deaths22,
            'Deaths23',Deaths23,
            'Deaths24',Deaths24,
            'Deaths25',Deaths25,
            'Deaths26',Deaths26,
            'IM_Deaths1',IM_Deaths1,
            'IM_Deaths2',IM_Deaths2,
            'IM_Deaths3',IM_Deaths3,
            'IM_Deaths4',IM_Deaths4
            ) as (age,count_deaths)
    from mortality
    where year >= '2010'
    """

df_mortality = spark.sql(query)
df_mortality.show()

+----+---+-------+-----+--------+------------+
|year|sex|country|cause|     age|count_deaths|
+----+---+-------+-----+--------+------------+
|2010|  1|   1400| 1000| Deaths1|         363|
|2010|  1|   1400| 1000| Deaths2|          12|
|2010|  1|   1400| 1000| Deaths3|           0|
|2010|  1|   1400| 1000| Deaths4|        null|
|2010|  1|   1400| 1000| Deaths5|        null|
|2010|  1|   1400| 1000| Deaths6|        null|
|2010|  1|   1400| 1000| Deaths7|           0|
|2010|  1|   1400| 1000| Deaths8|        null|
|2010|  1|   1400| 1000| Deaths9|           6|
|2010|  1|   1400| 1000|Deaths10|        null|
|2010|  1|   1400| 1000|Deaths11|          21|
|2010|  1|   1400| 1000|Deaths12|        null|
|2010|  1|   1400| 1000|Deaths13|          28|
|2010|  1|   1400| 1000|Deaths14|        null|
|2010|  1|   1400| 1000|Deaths15|          51|
|2010|  1|   1400| 1000|Deaths16|        null|
|2010|  1|   1400| 1000|Deaths17|          62|
|2010|  1|   1400| 1000|Deaths18|        null|
|2010|  1|   

In [11]:
df_mortality.createOrReplaceTempView("mortality_final")

In [12]:
spark.sql("""select year, 
            sum(count_deaths) 
            from mortality_final 
            where age != 'Deaths1' 
            group by year""").show()

+----+-----------------+
|year|sum(count_deaths)|
+----+-----------------+
|2012|         40309512|
|2010|         41544392|
|2011|         41040152|
|2015|         42103048|
|2013|         39123894|
|2014|         42610327|
|2016|         40957519|
|2018|         38348769|
|2019|         37369230|
|2020|          9237454|
|2017|         39406145|
+----+-----------------+



I had to hardcode field decodings from documentation since there's no datasources available with this data
1) See the documentation from WHO
2) http://www.wolfbane.com/icd/icd10a1.htm for the codes I didn't had

In [13]:
query = """
    select  
    m.year,
    case when m.sex = 1 then 'Male'
        when m.sex = 2 then 'Female'
        else 'No Data'
        end as sex,
    c.name as country,
    m.cause as cause_death_code,
    m.age,
    case 
        when m.age = 'Deaths1' then 'All ages'
        when m.age = 'Deaths2' then '0 year'
        when m.age = 'Deaths3' then '1 year'
        when m.age = 'Deaths4' then '2 years'
        when m.age = 'Deaths5' then '3 years'
        when m.age = 'Deaths6' then '4 years'
        when m.age = 'Deaths7' then '5-9 years'
        when m.age = 'Deaths8' then '10-14 years'
        when m.age = 'Deaths9' then '15-19 years'
        when m.age = 'Deaths10' then '20-24 years'
        when m.age = 'Deaths11' then '25-29 years'
        when m.age = 'Deaths12' then '30-34 years'
        when m.age = 'Deaths13' then '35-39 years'
        when m.age = 'Deaths14' then '40-44 years'
        when m.age = 'Deaths15' then '45-49 years'
        when m.age = 'Deaths16' then '50-54 years'
        when m.age = 'Deaths17' then '55-59 years'
        when m.age = 'Deaths18' then '60-64 years'
        when m.age = 'Deaths19' then '65-69 years'
        when m.age = 'Deaths20' then '70-74 years'
        when m.age = 'Deaths21' then '75-79 years'
        when m.age = 'Deaths22' then '80-84 years'
        when m.age = 'Deaths23' then '85-89 years'
        when m.age = 'Deaths24' then '90-94 years'
        when m.age = 'Deaths25' then '95 years and above' 
        when m.age = 'Deaths26' then 'Unspecified'
        when m.age = 'IM_deaths1' then 'Infant 0 day'
        when m.age = 'IM_deaths2' then 'Infant 1-6 days'
        when m.age = 'IM_deaths3' then 'Infant 7-27 days'
        when m.age = 'IM_deaths4' then 'Infant 28-364 days'
        end as age_range,
    coalesce(m.count_deaths,0) as count_deaths
    from mortality_final m
    left join country_codes c 
    on m.country = c.country
    where m.age != 'Deaths1'
    and ( substr(m.cause,1,1) in ('V','W','X','Y')
    or m.cause in ('1095','1096','1097','1098','1099','1100','1101','1102','1103'))
    and m.count_deaths >= 100
    """

df_final = spark.sql(query)
df_final.createOrReplaceTempView("final")
df_final.show()

+----+------+-------+----------------+--------+-----------+------------+
|year|   sex|country|cause_death_code|     age|  age_range|count_deaths|
+----+------+-------+----------------+--------+-----------+------------+
|2010|  Male|Belarus|            1095| Deaths9|15-19 years|         251|
|2010|  Male|Belarus|            1095|Deaths10|20-24 years|         617|
|2010|  Male|Belarus|            1095|Deaths11|25-29 years|         678|
|2010|  Male|Belarus|            1095|Deaths12|30-34 years|         809|
|2010|  Male|Belarus|            1095|Deaths13|35-39 years|         962|
|2010|  Male|Belarus|            1095|Deaths14|40-44 years|        1122|
|2010|  Male|Belarus|            1095|Deaths15|45-49 years|        1486|
|2010|  Male|Belarus|            1095|Deaths16|50-54 years|        1625|
|2010|  Male|Belarus|            1095|Deaths17|55-59 years|        1190|
|2010|  Male|Belarus|            1095|Deaths18|60-64 years|         932|
|2010|  Male|Belarus|            1095|Deaths19|65-6

In [14]:
df_final.createOrReplaceTempView("final_table")
spark.sql("""select count(*) from final_table""").show()

+--------+
|count(1)|
+--------+
|   20971|
+--------+



In [15]:
spark.sql("select  substr(cause_death_code,1,1), count(*) from final_table group by substr(cause_death_code,1,1)").show()

+------------------------------+--------+
|substr(cause_death_code, 1, 1)|count(1)|
+------------------------------+--------+
|                             Y|    1439|
|                             V|    3243|
|                             X|    8542|
|                             W|    3592|
|                             1|    4155|
+------------------------------+--------+



Save .csv to use in Tableau :D

In [16]:
df_final.toPandas().to_csv("data/External_mortality_causes.csv")